In [1]:
!pip install transformers==4.31.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 22.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [2]:
from transformers import CamembertTokenizer,CamembertModel

tokenizer = CamembertTokenizer.from_pretrained('camembert-base')
model = CamembertModel.from_pretrained('camembert-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

In [3]:
import pandas as pd

# Chemin vers le fichier Excel du corpus
chemin_corpus_pertinent = "/content/drive/MyDrive/corpus pertinent.xlsx"

# Charger le corpus depuis le fichier Excel
corpus_pertinent_df = pd.read_excel(chemin_corpus_pertinent)

# Extraire la colonne contenant les textes de votre DataFrame (par exemple, 'Texte' comme nom de colonne)
corpus_texts = corpus_pertinent_df['Texte'].tolist()  # Convertir en liste de textes

# Afficher quelques textes pour vérifier
print(corpus_texts[:5])  # Afficher les 5 premiers textes

['Spiruline de Nayalgué : Le Burkina à l’ère de la production industrielle.lundi 31 mai 2010 à 03h54min.Le Premier ministre, Tertius Zongo a présidé, le vendredi 28 mai 2010, à Nayalgué, localité située à quelques kilomètres de Koudougou, l’inauguration d’une unité de production de spiruline. Il en a profité pour autoriser la mise officielle sur le marché du FACA, un médicament pour le traitement de la drépanocytose.  Réalisée par le projet Spiruline Nayalgué, l’unité de production qui vient d’être inaugurée est le fruit d’un partenariat réussi entre le diocèse de Koudougou, l’ONG Technap (France) et le gouvernement du Burkina Faso.  Entièrement moderne, l’unité comprend un bâtiment technique, des bureaux et laboratoires équipés, un champ solaire de 128 plaques, un local technique, 2400 m2 de bassins aménagés en modules de production, un château-d’eau, deux puits busés, une route d’accès, un parking auto/moto et deux allées pavées.  Pour le ministre de la Santé, Seydou Bouda, cette uni

In [12]:
import nltk
nltk.download('punkt')

def tokenize_text(text):
  tokens = nltk.word_tokenize(text)
  return tokens

corpus_tokenized = [tokenize_text(text) for text in corpus_texts]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
#Suprression des caracteres speciaux
import re

def remove_special_chars(tokens):
  pattern = r'[^a-zA-Z0-9\s]'
  tokens = [re.sub(pattern, '', word) for word in tokens]
  return tokens

corpus_cleaned = [remove_special_chars(tokens) for tokens in corpus_tokenized]

In [15]:
#suppresion des mots vide
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('french'))

def remove_stopwords(tokens):
  tokens = [word for word in tokens if word.lower() not in stop_words]
  return tokens

corpus_no_stopwords = [remove_stopwords(tokens) for tokens in corpus_cleaned]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
#extrair les embedding
from transformers import CamembertTokenizer,CamembertModel
import torch

tokenizer = CamembertTokenizer.from_pretrained('camembert-base')
model = CamembertModel.from_pretrained('camembert-base')

encoded_corpus = tokenizer([' '.join(text) for text in corpus_no_stopwords], return_tensors='pt', padding=True, truncation=True)

with torch.no_grad():
  outputs = model(**encoded_corpus)
  embeddings = outputs.last_hidden_state

mean_embeddings = torch.mean(embeddings, dim=1)

global_embedding = torch.mean(mean_embeddings, dim=0)

from sklearn.metrics.pairwise import cosine_similarity

cosine_similarities = cosine_similarity(mean_embeddings, global_embedding.unsqueeze(0).numpy())

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [17]:
terms = []
for i, text in enumerate(corpus_no_stopwords):
  tokens_with_scores = list(zip(text, cosine_similarities[i]))
  tokens_with_scores_sorted = sorted(tokens_with_scores, key=lambda x: x[1], reverse=True)
  terms.append([token for token, score in tokens_with_scores_sorted if score > 0.8]) # Exemple de seuil : 0.8

print(terms)

[['Spiruline'], ['RICHE'], ['Site'], [], ['Ral'], [], ['Atteinte'], ['7'], [], ['Orphelinat'], ['Ecole'], ['Site'], ['Inauguration'], [], [], ['Carnet'], ['Affaire'], ['Fait'], ['Campagne'], ['Insolite'], ['Pluie'], [], ['CONSTRUCTION'], ['Civisme'], ['Accident'], ['Baccalaurat'], ['Campagne'], ['CONSTRUCTION'], ['11Dcembre'], [], [], [''], [], ['Inauguration'], ['Village'], ['Infrastructures'], ['Inondations'], ['Village'], ['Accident'], ['Dveloppement'], ['8Mars'], ['Zinigma'], ['station']]


In [18]:
import pandas as pd

terms_with_scores = []
for i, text in enumerate(corpus_no_stopwords):
  tokens_with_scores = list(zip(text, cosine_similarities[i]))
  for token, score in tokens_with_scores:
    terms_with_scores.append([token, score])

df = pd.DataFrame(terms_with_scores, columns=['Terme', 'Score de similarité'])
print(df)

              Terme  Score de similarité
0         Spiruline             0.947732
1             RICHE             0.942857
2              Site             0.947398
3         Abattages             0.736067
4               Ral             0.946695
5             Crash             0.574887
6          Atteinte             0.859067
7                 7             0.853721
8             Socit             0.677368
9        Orphelinat             0.911008
10            Ecole             0.955212
11             Site             0.872835
12     Inauguration             0.950251
13            Manga             0.666019
14          Burkina             0.540479
15           Carnet             0.940519
16          Affaire             0.935734
17             Fait             0.815129
18         Campagne             0.952483
19         Insolite             0.924518
20            Pluie             0.934531
21         Rsultats             0.435767
22     CONSTRUCTION             0.944355
23          Civi

In [19]:
df = df[~df['Terme'].str.isnumeric()]

In [20]:
df

,Terme,Score de similarité
0,Spiruline,0.947732
1,RICHE,0.942857
2,Site,0.947398
3,Abattages,0.736067
4,Ral,0.946695
5,Crash,0.574887
6,Atteinte,0.859067
8,Socit,0.677368
9,Orphelinat,0.911008
10,Ecole,0.955212


In [23]:
df = df.dropna(subset=['Terme'])

In [24]:
# Enregistrer le DataFrame dans un fichier Excel
df.to_excel("termes_et_scores.xlsx", index=False)

# Télécharger le fichier (Code pour Colab)
from google.colab import files
files.download('termes_et_scores.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>